# CSE 280A Project

In [1]:
import pandas as pd
import numpy as np

In [3]:
## Load the reference file (classification for each sample)
ref = pd.read_csv("../data/pcaa-master-platinum_set_2020.tsv", sep="\t", header=0, usecols=[0,7,8,12,13,28,32])
print(ref.shape)
ref.head()

(3212, 7)


,study,patient_barcode,sample_barcode,min_cnv,number_amp_region,sample_classification_nam_2019,sample_classification
0,PCWG,DO51086,SA530559,4,30,NaN,Non-circular
1,PCWG,DO48759,SA515463,4,5,NaN,Non-circular
2,PCWG,DO48916,SA517420,4,0,NaN,No SCNA detected
3,PCWG,DO51046,SA530430,4,2,NaN,No SCNA detected
4,PCWG,DO45251,SA501507,4,3,NaN,Circular


In [4]:
## Only keep TCGA samples
ref = ref[ref['study'] == 'TCGA']
print(ref.shape)
ref.head()

(1921, 7)


,study,patient_barcode,sample_barcode,min_cnv,number_amp_region,sample_classification_nam_2019,sample_classification
1291,TCGA,TCGA-02-2483,TCGA-02-2483-01,1,20,Distal,Heavily-rearranged
1292,TCGA,TCGA-02-2485,TCGA-02-2485-01,1,33,Circular,Circular
1293,TCGA,TCGA-04-1331,TCGA-04-1331-01,1,4,Linear,Non-circular
1294,TCGA,TCGA-04-1347,TCGA-04-1347-01,1,12,Circular,Circular
1295,TCGA,TCGA-04-1349,TCGA-04-1349-01,1,26,Distal,Heavily-rearranged


In [5]:
## Load the oncoprint matrix after querying genes against all TCGA studies
res = pd.read_csv("../data/PATIENT_DATA_oncoprint.tsv", sep="\t", header=0, skiprows=[1,2,3,4])
print(res.shape)
res.head()

(315, 10955)


,track_name,track_type,TCGA-ER-A195,TCGA-XK-AAJA,TCGA-CH-5788,TCGA-OR-A5JX,TCGA-CJ-4886,TCGA-EQ-A4SO,TCGA-CE-A3MD,TCGA-DM-A28K,...,TCGA-97-A4M2,TCGA-97-7552,TCGA-97-8552,TCGA-98-A53C,TCGA-98-A53D,TCGA-98-A53H,TCGA-99-AA5R,TCGA-13-2066,TCGA-BP-4345,TCGA-OR-A5OG
0,APC,CNA,homdel_rec,homdel_rec,homdel_rec,Amplification,Amplification,NaN,NaN,homdel_rec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARHGEF12,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATM,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BCL11B,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BLM,CNA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
## Additional information of the oncoprint matrix
info = pd.read_csv("../data/PATIENT_DATA_oncoprint.tsv", sep="\t", header=0, skiprows=lambda x: x not in range(5))
print(info.shape)
info.head()

(4, 10955)


,track_name,track_type,TCGA-ER-A195,TCGA-XK-AAJA,TCGA-CH-5788,TCGA-OR-A5JX,TCGA-CJ-4886,TCGA-EQ-A4SO,TCGA-CE-A3MD,TCGA-DM-A28K,...,TCGA-97-A4M2,TCGA-97-7552,TCGA-97-8552,TCGA-98-A53C,TCGA-98-A53D,TCGA-98-A53H,TCGA-99-AA5R,TCGA-13-2066,TCGA-BP-4345,TCGA-OR-A5OG
0,Study of origin,CLINICAL,"Skin Cutaneous Melanoma (TCGA, PanCancer Atlas)","Prostate Adenocarcinoma (TCGA, PanCancer Atlas)","Prostate Adenocarcinoma (TCGA, PanCancer Atlas)","Adrenocortical Carcinoma (TCGA, PanCancer Atlas)","Kidney Renal Clear Cell Carcinoma (TCGA, PanCa...","Stomach Adenocarcinoma (TCGA, PanCancer Atlas)","Thyroid Carcinoma (TCGA, PanCancer Atlas)","Colorectal Adenocarcinoma (TCGA, PanCancer Atlas)",...,"Lung Adenocarcinoma (TCGA, PanCancer Atlas)","Lung Adenocarcinoma (TCGA, PanCancer Atlas)","Lung Adenocarcinoma (TCGA, PanCancer Atlas)","Lung Squamous Cell Carcinoma (TCGA, PanCancer ...","Lung Squamous Cell Carcinoma (TCGA, PanCancer ...","Lung Squamous Cell Carcinoma (TCGA, PanCancer ...","Lung Adenocarcinoma (TCGA, PanCancer Atlas)","Ovarian Serous Cystadenocarcinoma (TCGA, PanCa...","Kidney Renal Clear Cell Carcinoma (TCGA, PanCa...","Adrenocortical Carcinoma (TCGA, PanCancer Atlas)"
1,# Samples per Patient,CLINICAL,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,Profiled for copy number alterations,CLINICAL,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,NaN,NaN,NaN
3,Profiled for mutations,CLINICAL,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,Yes,Yes,Yes,Yes,NaN,Yes,Yes,NaN,NaN,NaN


In [7]:
## Filter samples out - only keep samples/patients with ecDNA classification
res = res[['track_name'] + list(res.columns[res.columns.isin(ref['patient_barcode'])])]
print(res.shape)
res.head()


(315, 1901)


,track_name,TCGA-CH-5788,TCGA-FS-A1ZG,TCGA-DD-A3A8,TCGA-AD-6964,TCGA-A2-A0EY,TCGA-EJ-7784,TCGA-EJ-5531,TCGA-CH-5748,TCGA-AG-4008,...,TCGA-VD-A8K8,TCGA-VD-A8KA,TCGA-VD-A8KD,TCGA-VD-A8KE,TCGA-VD-A8KF,TCGA-VD-A8KH,TCGA-VD-A8KL,TCGA-VD-A8KN,TCGA-VD-AA8O,TCGA-VD-AA8P
0,APC,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARHGEF12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BCL11B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BLM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
## Group the columns into ecDNA+ vs ecDNA-
## Strategy 1: ecDNA- = Non-circular; ecDNA+ = otherwise
ecdna_neg = ref[ref['sample_classification'] == 'Non-circular']['patient_barcode']
ecdna_pos = ref[ref['sample_classification'] != 'Non-circular']['patient_barcode']
pos1 = list(res.columns[res.columns.isin(ecdna_pos)])
print("Oncoprint1: {}, {}".format(pos1[-1], len(pos1)))
cols_met1 = ['track_name'] + pos1 + list(res.columns[res.columns.isin(ecdna_neg)])
df1 = res[cols_met1]
print(df1.shape)
df1.head()

Oncoprint1: TCGA-VD-AA8P, 1735
(315, 1901)


,track_name,TCGA-FS-A1ZG,TCGA-DD-A3A8,TCGA-AD-6964,TCGA-A2-A0EY,TCGA-EJ-7784,TCGA-EJ-5531,TCGA-CH-5748,TCGA-D3-A2JC,TCGA-68-7755,...,TCGA-EK-A3GJ,TCGA-FU-A3NI,TCGA-G3-A25W,TCGA-IR-A3LC,TCGA-KL-8328,TCGA-RZ-AB0B,TCGA-V4-A9E7,TCGA-V4-A9EE,TCGA-V4-A9EU,TCGA-V4-A9F3
0,APC,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARHGEF12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BCL11B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BLM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
## Save this oncoprint matrix
df1.to_csv("../data/oncoprint_1.tsv", sep="\t", index=False)

In [19]:
## Strategy 2: ecDNA+ = Circular; ecDNA- = otherwise
ecdna_pos = ref[ref['sample_classification'] == 'Circular']['patient_barcode']
ecdna_neg = ref[ref['sample_classification'] != 'Circular']['patient_barcode']
pos2 = list(res.columns[res.columns.isin(ecdna_pos)])
print("Oncoprint2: {}, {}".format(pos2[-1], len(pos2)))
cols_met2 = ['track_name'] + pos2 + list(res.columns[res.columns.isin(ecdna_neg)])
df2 = res[cols_met1]
print(df2.shape)
df2.head()

Oncoprint2: TCGA-G2-A2EK, 335
(315, 1901)


,track_name,TCGA-FS-A1ZG,TCGA-DD-A3A8,TCGA-AD-6964,TCGA-A2-A0EY,TCGA-EJ-7784,TCGA-EJ-5531,TCGA-CH-5748,TCGA-D3-A2JC,TCGA-68-7755,...,TCGA-EK-A3GJ,TCGA-FU-A3NI,TCGA-G3-A25W,TCGA-IR-A3LC,TCGA-KL-8328,TCGA-RZ-AB0B,TCGA-V4-A9E7,TCGA-V4-A9EE,TCGA-V4-A9EU,TCGA-V4-A9F3
0,APC,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,homdel_rec,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARHGEF12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BCL11B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BLM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df2.to_csv("../data/oncoprint_2.tsv", sep="\t", index=False)